In [ ]:
import numpy as np
import pandas as pd

# === PREPARAÇÃO DOS DADOS - REGRESSÃO (Prever Qualidade) ===
data = pd.read_csv('winequality_bruto.csv')

X = data.drop(['quality', 'classe'], axis=1).values # Todas as colunas numéricas, menos 'quality' e 'classe'
y = data['quality'].values.reshape(-1, 1) # Target contínuo; .reshape utilizado para criar uma matriz X

# === NORMALIZAÇÃO ===
X_media = X.mean(axis=0)
X_std = X.std(axis=0)
X_std[X_std == 0] = 1
X_norm = (X - X_media) / X_std


#=== FUNÇÕES DO MODELO ===
'''
- pred: multiplicação dos vetores (X, w)
- mse: erro quadrático médio
- rmse: erro quadrático médio na unidade de y
- mae: erro absoluto médio (mais robusto)
- r²: coeficiente de determinação
- train: gradiente descendente
'''
def pred(X, w, b):
    return np.dot(X, w) + b

def mse(y_true, y_pred):
    return np.mean((y_true - y_pred) ** 2)

def rmse(y_true, y_pred):
    return np.sqrt(mse(y_true, y_pred))

def mae(y_true, y_pred):
    return np.mean(np.abs(y_true - y_pred))

def r2_score(y_true, y_pred):
    ss_res = np.sum((y_true - y_pred) ** 2)
    ss_tot = np.sum((y_true - np.mean(y_true)) ** 2)
    return 1 - (ss_res / ss_tot)

def train(X, y, lr=0.01, epochs=1000):
    m, n = X.shape              #
    w = np.zeros((n, 1))        # Palpites iniciais
    b = 0                       #
    for i in range(epochs):
        y_pred = pred(X, w, b)
        error = y_pred - y
        dw = (1/m) * np.dot(X.T, error) # Derivada da função de custo J(w,b) em relação a w
        db = (1/m) * np.sum(error) # Derivada da função de custo J(w,b) em relação a b
        w -= lr * dw
        b -= lr * db # Atualização dos palpites por época + taxa de aprendizado
        # Um passo na direção oposta ao gradiente controlado pela taxa de aprendizado = Se a derivada dw for muito positiva,
        # a atualização é muito negativa (visando "concertar e miniizar o erro") (...) sucessivamente
    return w, b

# === TREINAMENTO ===
w, b = train(X_norm, y)
y_pred = pred(X_norm, w, b)

# === MÉTRICAS DE AVALIAÇÃO ===
mse_value = mse(y, y_pred)
rmse_value = rmse(y, y_pred)
mae_value = mae(y, y_pred)
r2_value = r2_score(y, y_pred)

print("=== MÉTRICAS DE REGRESSÃO ===")
print(f"MSE  = {mse_value:.4f}")
print(f"RMSE = {rmse_value:.4f}")
print(f"MAE  = {mae_value:.4f}")
print(f"R²   = {r2_value:.4f}")

=== MÉTRICAS DE REGRESSÃO ===
MSE  = 0.5401
RMSE = 0.7349
MAE  = 0.5688
R²   = 0.2916


In [7]:
import numpy as np
import pandas as pd

# === PREPARAÇÃO DOS DADOS - CLASSIFICAÇÃO (Prever Classe) ===
data = pd.read_csv('winequality_bruto.csv')
data['classe_b'] = data['classe'].apply(lambda x: 1 if x == 'branco' else 0) # Conversão do texto em binário (1 = branco, 0 = tinto)
X = data.drop(['quality', 'classe', 'classe_b'], axis=1).values # Todas as colunas numéricas, menos 'quality','classe' e 'classe_b'
y = data['classe_b'].values.reshape(-1, 1)  # Target binário; .reshape utilizado para criar uma matriz X

# === NORMALIZAÇÃO ===
X_media = X.mean(axis=0)
X_std = X.std(axis=0)
X_std[X_std == 0] = 1
X_norm = (X - X_media) / X_std


#=== FUNÇÕES DO MODELO ===
'''
- sigmoide: função da regressão logística
- prpb: predição 
- predict: condição da predição >= 0.5
- cross_entropy: 
- accuracy: acurácia
- precision: precisão
- recall
- f1_score
- train_logistic: gradiente descendente
'''
def sigmoide(z):
    return 1 / (1 + np.exp(-z))

def prpb(X, w, b):
    return sigmoide(np.dot(X, w) + b)

def predict(X, w, b):
    return (prpb(X, w, b) >= 0.5)

def cross_entropy(y_true, y_pred):
    eps = 1e-10
    y_pred = np.clip(y_pred, eps, 1 - eps)
    return -np.mean(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))

def accuracy(y_true, y_pred_class):
    return np.mean(y_true == y_pred_class)

def precision(y_true, y_pred):
    tp = np.sum((y_true == 1) & (y_pred == 1))
    fp = np.sum((y_true == 0) & (y_pred == 1))
    return tp / (tp + fp + 1e-10)

def recall(y_true, y_pred):
    tp = np.sum((y_true == 1) & (y_pred == 1))
    fn = np.sum((y_true == 1) & (y_pred == 0))
    return tp / (tp + fn + 1e-10)

def f1_score(y_true, y_pred):
    p = precision(y_true, y_pred)
    r = recall(y_true, y_pred)
    return 2 * p * r / (p + r + 1e-10)

def train_logistic(X, y, lr=0.01, epochs=1000):
    m, n = X.shape
    w = np.zeros((n, 1))
    b = 0
    for i in range(epochs):
        y_pred = prpb(X, w, b)
        error = y_pred - y
        dw = (1/m) * np.dot(X.T, error)
        db = (1/m) * np.sum(error)
        w -= lr * dw
        b -= lr * db
    return w, b

# === TREINAMENTO ===
w, b = train_logistic(X_norm, y)
y_pred_prob = prpb(X_norm, w, b)
y_pred_class = predict(X_norm, w, b)

# === MÉTRICAS DE AVALIAÇÃO ===
cross_value = cross_entropy(y, y_pred_prob)
acc_value = accuracy(y, y_pred_class)
prec_value = precision(y, y_pred_class)
rec_value = recall(y, y_pred_class)
f1_value = f1_score(y, y_pred_class)

# --- Exibir métricas ---
print("=== MÉTRICAS DE CLASSIFICAÇÃO ===")
print(f"Loss final (Cross-Entropy): {cross_value:.4f}")
print(f"Acurácia: {acc_value:.4f}")
print(f"Precisão: {prec_value:.4f}")
print(f"Recall:   {rec_value:.4f}")
print(f"F1-Score: {f1_value:.4f}")

=== MÉTRICAS DE CLASSIFICAÇÃO ===
Loss final (Cross-Entropy): 0.1246
Acurácia: 0.9854
Precisão: 0.9959
Recall:   0.9847
F1-Score: 0.9902
